<a href="https://colab.research.google.com/github/Matiasballesta/Deploy-PI/blob/master/BLOCKCHAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask==0.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.1 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
!pip install flask-ngrok==0.0.25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importacion de librerias
import datetime
import hashlib
import json
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
# Creación de la Blockchain
class Blockchain:
    
  def __init__(self):
    """ Constructor de la clase Blockchain. """

    self.chain = []
    self.create_block(proof = 1, previous_hash = '0')
      
  
  def create_block(self, proof, previous_hash):
    """ Creación de un nuevo bloque. 

      Arguments:
        - proof: Nounce del bloque actual. (proof != hash)
        - previous_hash: Hash del bloque previo.

      Returns: 
        - block: Nuevo bloque creado. 
      """

    block = { 'index'         : len(self.chain)+1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash}
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """ Obtención del bloque previo de la Blockchain .
    
      Returns:
        - Obtención del último bloque de la Blockchain. """

    return self.chain[-1]
  
  def proof_of_work(self, previous_proof):
    """ Protocolo de concenso Proof of Work (PoW).
    
      Arguments:
        - previous_proof: Nounce del bloque previo.

      Returns:
        - new_proof: Devolución del nuevo nounce obtenido con PoW. """

    new_proof = 1
    check_proof = False
    while check_proof is False:
        hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] == '0000':
            check_proof = True
        else: 
            new_proof += 1
    return new_proof
  
  def hash(self, block):
    """ Cálculo del hash de un bloque.
    
    Arguments:
        - block: Identifica a un bloque de la Blockchain.
    
    Returns:
        - hash_block: Devuelve el hash del bloque """

    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block
  
  def is_chain_valid(self, chain):
    """ Determina si la Blockchain es válida. 
    
    Arguments:
        - chain: Cadena de bloques que contiene toda la 
                  información de las transacciones.
    
    Returns:
        - True/False: Devuelve un booleano en función de la validez de la 
                      Blockchain. (True = Válida, False = Inválida) """

    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True

In [ ]:
# Crear una aplicación web
# Ejecución de la app con Notebook
app = Flask(__name__)
run_with_ngrok(app)  

# Si se obtiene un error 500, actualizar flask y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Creación de la Blockchain
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ Minado de un nuevo bloque """

  previous_block  = blockchain.get_previous_block()
  previous_proof  = previous_block['proof']
  proof           = blockchain.proof_of_work(previous_proof)
  previous_hash   = blockchain.hash(previous_block)
  block           = blockchain.create_block(proof, previous_hash)
  response = {'message'       : '¡Enhorabuena, has minado un nuevo bloque!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash']}
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Obtención de la Blockchain """
  response = {'chain'   : blockchain.chain, 
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación si la Blockchain es válida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : '¡La cadena de bloques es válida!'}
  else:
      response = {'message' : '¡La cadena de bloques NO es válida!'}
  return jsonify(response), 200

In [6]:
# Ejecución de la app con Google Colab
app.run()

# Ejecución externa a Google colab
# app.run(host = '0.0.0.0', port = 5000)

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://19df-34-148-60-80.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:45:16] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:45:18] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:46:05] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:47:48] "GET /get_chain HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:20] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:28] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:30] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:31] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:32] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:33] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:34] "GET /mine_block HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Sep/2022 16:48:56] "GET /get_chain HTTP/1.1" 200 -
INFO:werkzeug: